In [1]:
## 更新股票代碼
import twstock
twstock.__update_codes()

## 擷取上市股票代碼
import twstock
import os
twse_dict = twstock.twse
twse_code0 = twse_dict.keys()
twse_code = []
for i in twse_code0:
    if len(i) == 4:
        twse_code.append(i)

## 擷取上櫃股票代碼
tpex_dict = twstock.tpex
tpex_code0 = tpex_dict.keys()
tpex_code = []
for i in tpex_code0:
    if len(i) == 4:
        tpex_code.append(i)
        
print('上市公司家數: ' + str(len(twse_code)))       
print('上櫃公司家數: ' + str(len(tpex_code)))

## 回到原始路徑
# os.chdir('D:\\WEI\\TWStock')

上市公司家數: 973
上櫃公司家數: 789


In [2]:
## 儲存參數程式
import pickle

def save_variable(v,filename):
  f=open(filename,'wb')
  pickle.dump(v,f)
  f.close()
  return filename

def load_variavle(filename):
  f=open(filename,'rb')
  r=pickle.load(f)
  f.close()
  return r

# filename = save_variable(results,'results.txt')  ##儲存參數
# results = load_variavle('results.txt')  ##讀取參數

In [3]:
## 計算個股價、量、ma(該程式碼設定近200個交易日)
## 注意!! output時間序顛倒
import pandas_datareader as pdr
def ma_count(stock,twse,start,end):
    ##==========================##
    # input
    # stock : 股票代碼
    # twse : 是否為上市公司。是(1)；不是(0)
    # start : 擷取股市資料起始日期
    # end : 擷取股市資料終止日期
    
    if twse == 1: #是上市公司
        stock_inf = pdr.DataReader(stock+'.TW', 'yahoo',start,end)
    else:         #是上櫃公司
        stock_inf = pdr.DataReader(stock+'.TWO', 'yahoo',start,end)

    price = list(stock_inf['Close'])[::-1]           ## 擷取收盤價
    volume = list((stock_inf['Volume'])/1000)[::-1]
    stock_reslut = []
    ## 擷取近200個交易日
    for i in range(200):
        p5 = []
        p10 = []
        p20 = []
        p60 = []
        v5 = []
        v10 =[]

        for j in range(5):
            j = i + j
            p5.append(price[j])
            v5.append(volume[j])
        for j in range(10):
            j = i + j
            p10.append(price[j])
            v10.append(volume[j])
        for j in range(20):
            j = i + j
            p20.append(price[j])
        for j in range(60):
            j = i + j
            p60.append(price[j])

        price_n = price[i]
        volume_n = int(volume[i])
        ma5 = sum(p5)/5
        ma10 = sum(p10)/10
        ma20 = sum(p20)/20
        ma60 = sum(p60)/60
        vma5 = sum(v5)/5
        vma10 = sum(v10)/10

        stock_r = [price_n,volume_n,ma5,ma10,ma20,ma60,vma5,vma10]
        stock_reslut.append(stock_r)

    return stock_reslut

## 注意!! output時間序顛倒

In [4]:
## 檢視價、量、ma關係
def stock_check(stock_reslut):
    
    check_result = []
    ## 檢視當天股價與ma之關係，以及成交量與vma之關係，最後檢視近10日平均成交量是否大於500張
    for i in stock_reslut:
        stock_n = i            #個股當天資訊list
        price_n = stock_n[0]   #當天股價
        volume_n = stock_n[1]  #當天成交量
        ma5 = stock_n[2]       #當天5ma
        ma10 = stock_n[3]      #當天10ma
        ma20 = stock_n[4]      #當天20ma
        ma60 = stock_n[5]      #當天60ma
        vma5 = stock_n[6]      #當天5vma
        vma10 = stock_n[7]     #當天10vma
        
        chk_ma5 = (price_n - ma5)/ma5 * 100      #與均線距離(%) ma5
        chk_ma10 = (price_n - ma10)/ma10 * 100   #與均線距離(%) ma10
        chk_ma20 = (price_n - ma20)/ma20 * 100   #與均線距離(%) ma20
        chk_ma60 = (price_n - ma60)/ma60 * 100   #與均線距離(%) ma60
        
        if vma5 == 0:
            vma5 = 1
        if vma10 == 0:
            vma10 = 1
        chk_vma5 = (volume_n - vma5)/vma5 * 100     #與成交量均線距離(%) vma5
        chk_vma10 = (volume_n - vma10)/vma10 * 100  #與成交量均線距離(%) vma10
        
        check_r = [price_n,volume_n,chk_ma5,chk_ma10,chk_ma20,chk_ma60,chk_vma5,chk_vma10]
        check_result.append(check_r)
    return check_result

In [5]:
### 主程式 ###
## 設定爬蟲時間段
import datetime
start = datetime.datetime.now() - datetime.timedelta(days=500)
end = datetime.date.today()
print(end)

## 上市公司全部成果(價、量、ma)
stock_error = []
twse_stock_result = {}
twse_check_result = {}
for i in twse_code:
    try:
        stock_result = ma_count(i,1,start,end)
        twse_stock_result[i] = stock_result
        check_result = stock_check(stock_result)
        twse_check_result[i] = check_result  
    except:
        print('Error : ' + 'TWSE--' + str(i))
        stock_error.append(i)
    

## 上櫃公司全部成果(價、量、ma)
tpex_stock_result = {}
tpex_check_result = {}
for i in tpex_code:
    try:
        stock_result = ma_count(i,0,start,end)
        tpex_stock_result[i] = stock_result
        check_result = stock_check(stock_result)
        tpex_check_result[i] = check_result    
    except:
        print('Error : ' + 'TPEX--' + str(i))
        stock_error.append(i)
    
    
## 儲存上市上櫃全部stock成果到電腦
import os
patho = os.getcwd() 
os.chdir(os.getcwd() + '\stock_result')

pathw = os.getcwd() + '\\' + str(end)
if not os.path.isdir(pathw):
    os.mkdir(pathw)

os.chdir(pathw)
save_variable(twse_stock_result,'twse_stock_result.txt')
save_variable(tpex_stock_result,'tpex_stock_result.txt')
os.chdir(patho)

## 儲存上市上櫃全部check成果到電腦
patho = os.getcwd() 
os.chdir(os.getcwd() + '\check_result')

pathw = os.getcwd() + '\\' + str(end)
if not os.path.isdir(pathw):
    os.mkdir(pathw)

os.chdir(pathw)
save_variable(twse_check_result,'twse_check_result.txt')
save_variable(tpex_check_result,'tpex_check_result.txt')
os.chdir(patho)

## 儲存stock_error到電腦
patho = os.getcwd() 
os.chdir(os.getcwd() + '\stock_error')

pathw = os.getcwd() + '\\' + str(end)
if not os.path.isdir(pathw):
    os.mkdir(pathw)

os.chdir(pathw)
save_variable(stock_error,'stock_error.txt')
os.chdir(patho)

2021-12-11
Error : TWSE--2025
Error : TWSE--2250
Error : TWSE--3714
Error : TWSE--6768
Error : TWSE--6770
Error : TWSE--6781
Error : TWSE--6806
Error : TPEX--6741


In [24]:
print(stock_error)

['2025', '2250', '3714', '6768', '6781', '6806', '6741']


In [18]:
## 測試
import datetime
start = datetime.datetime.now() - datetime.timedelta(days=500)
end = datetime.date.today()

stock_result = ma_count('2331',1,start,end)
check_result = stock_check(stock_result)
print(check_result[0])

[23.350000381469727, 5567, -0.6382978619790871, 2.009612170447402, 2.1993668562589463, 0.33301240219448885, -24.368727491816756, -17.648955945712103]


In [25]:
twse_stock_result['2331']

[[23.700000762939453,
  5704,
  23.38000030517578,
  22.685000038146974,
  22.872500038146974,
  23.298333390553793,
  6973.287399999999,
  6934.4966],
 [23.700000762939453,
  6994,
  23.220000076293946,
  22.51499996185303,
  22.872500038146974,
  23.316666698455812,
  9504.7024,
  6859.1185],
 [23.600000381469727,
  8450,
  22.879999923706055,
  22.374999809265137,
  22.872500038146974,
  23.307500012715657,
  8700.8782,
  6867.4661],
 [23.149999618530273,
  10086,
  22.56999969482422,
  22.299999809265138,
  22.872500038146974,
  23.299166679382324,
  7575.8848,
  7090.1732],
 [22.75,
  3630,
  22.279999923706054,
  22.38499984741211,
  22.895000076293947,
  23.298333358764648,
  6159.4568,
  7054.284199999999],
 [22.899999618530273,
  18361,
  21.989999771118164,
  22.479999923706053,
  22.920000076293945,
  23.305833371480308,
  6895.705799999999,
  7241.051099999999],
 [22.0,
  2975,
  21.809999847412108,
  22.504999923706055,
  22.872500133514404,
  23.28916670481364,
  4213.534

In [13]:
import numpy as np

a = {'a':1,'b':2}
np.save("a.npy", a)
c = np.load( "a.npy", allow_pickle='TRUE')